In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

In [4]:
url="http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv"
original_data=pd.read_csv(url,error_bad_lines=False,sep='\t')

In [5]:
original_data.shape

(404290, 6)

In [6]:
original_data = original_data.dropna()

In [7]:
data = original_data

In [8]:
data.tail()

,id,qid1,qid2,question1,question2,is_duplicate
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0
404289,404289,537932,537933,What is like to have sex with cousin?,What is it like to have sex with your cousin?,0


In [9]:
columns = ['question1','question2']

In [56]:
data_sub = data[columns]

In [11]:
labels = data['is_duplicate']

In [12]:
import torch
import torch.nn as nn
import pickle
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam,BertForMaskedLM
from pytorch_pretrained_bert.modeling import PreTrainedBertModel, BertModel

Using TensorFlow backend.


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [14]:
import torch.nn.functional as F

In [15]:
bs = 32

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [17]:
n_gpu

3

In [18]:
!nvidia-smi

Tue Feb  5 23:47:28 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 387.26                 Driver Version: 387.26                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 30%   53C    P0    63W / 250W |     10MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:09:00.0 Off |                  N/A |
| 27%   49C    P0    62W / 250W |     10MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [54]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [57]:
for index,row in data_sub.iterrows():
    for column in columns:
        data_sub.at[index,column] = tokenizer.tokenize(row[column])

In [58]:
data_sub.tail()

,question1,question2
404285,"[how, many, key, ##words, are, there, in, the,...","[how, many, key, ##words, are, there, in, per,..."
404286,"[do, you, believe, there, is, life, after, dea...","[is, it, true, that, there, is, life, after, d..."
404287,"[what, is, one, coin, ?]","[what, ', s, this, coin, ?]"
404288,"[what, is, the, approx, annual, cost, of, livi...","[i, am, having, little, hair, ##fall, problem,..."
404289,"[what, is, like, to, have, sex, with, cousin, ?]","[what, is, it, like, to, have, sex, with, your..."


In [ ]:
with open('data_bert_preprocessed.pickle', 'wb') as sub_data:
    pickle.dump(data_sub, sub_data, protocol=pickle.HIGHEST_PROTOCOL)

In [103]:
with open('data_bert_preprocessed.pickle', 'rb') as handle:
    data_sub = pickle.load(handle)

In [60]:
MAX_LEN = max(data_sub.question1.map(lambda x: len(x)).max(),data_sub.question1.map(lambda x: len(x)).max())

In [61]:
MAX_LEN

153

In [63]:
for index,row in data_sub.iterrows():
    for column in columns:
        data_sub.at[index,column] = pad_sequences([tokenizer.convert_tokens_to_ids(row[column])],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [65]:
data_sub.tail()

,question1,question2
404285,"[[2129, 2116, 3145, 22104, 2024, 2045, 1999, 1...","[[2129, 2116, 3145, 22104, 2024, 2045, 1999, 2..."
404286,"[[2079, 2017, 2903, 2045, 2003, 2166, 2044, 23...","[[2003, 2009, 2995, 2008, 2045, 2003, 2166, 20..."
404287,"[[2054, 2003, 2028, 9226, 1029, 0, 0, 0, 0, 0,...","[[2054, 1005, 1055, 2023, 9226, 1029, 0, 0, 0,..."
404288,"[[2054, 2003, 1996, 22480, 3296, 3465, 1997, 2...","[[1045, 2572, 2383, 2210, 2606, 13976, 3291, 2..."
404289,"[[2054, 2003, 2066, 2000, 2031, 3348, 2007, 55...","[[2054, 2003, 2009, 2066, 2000, 2031, 3348, 20..."


In [67]:
with open('data_bert_integer.pickle', 'wb') as sub_data:
    pickle.dump(data_sub, sub_data, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
with open('data_bert_integer.pickle', 'rb') as handle:
    data_sub = pickle.load(handle)

In [18]:
#attention_masks1 = []
#attention_masks2 = []

In [29]:
#for index,row in data_sub.iterrows():
#            attention_masks1.append([float(i>0) for i in ii] for ii in row['question1'] )

In [30]:
#for index,row in data_sub.iterrows():
#            attention_masks2.append([float(i>0) for i in ii] for ii in row['question2'] )

In [31]:
#attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [36]:
tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(data_sub[:50000], labels[:50000], 
                                                            random_state=2018, test_size=0.1)
#tr_masks1, val_masks1,tr_masks2, val_masks2 = train_test_split(attention_masks1, attention_masks2,
#                                             random_state=2018, test_size=0.1)

In [37]:
tr_inputs1 = tr_inputs.question1
tr_inputs2 = tr_inputs.question2
val_inputs1 = val_inputs.question1
val_inputs2 = val_inputs.question2

In [38]:
import itertools
tr_inputs1 = list(itertools.chain(*tr_inputs1))
tr_inputs2 = list(itertools.chain(*tr_inputs2))
val_inputs1 = list(itertools.chain(*val_inputs1))
val_inputs2 = list(itertools.chain(*val_inputs2))

In [39]:
tr_tags = np.array(tr_tags)
val_tags = np.array(val_tags)

In [40]:
tr_inputs1 = torch.tensor(tr_inputs1)
tr_inputs2 = torch.tensor(tr_inputs2)
val_inputs1 = torch.tensor(val_inputs1)
val_inputs2 = torch.tensor(val_inputs2)
tr_labels = torch.tensor(tr_tags)
val_labels = torch.tensor(val_tags)
#tr_masks1 = torch.tensor(tr_masks1)
#val_masks1 = torch.tensor(val_masks1)
#tr_masks2 = torch.tensor(tr_masks2)
#val_masks2 = torch.tensor(val_masks2)

In [41]:
tr_labels.size()

torch.Size([45000])

In [42]:
train_data = TensorDataset(tr_inputs1, tr_inputs2, tr_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=16)

valid_data = TensorDataset(val_inputs1, val_inputs2, val_labels)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=16)

In [45]:
class BertForQuoraClassification(PreTrainedBertModel):

    def __init__(self, config, num_labels=1):
        super(BertForQuoraClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        #self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.apply(self.init_bert_weights)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
        _, pooled_output = self.bert(input_ids, token_type_ids, attention_mask, output_all_encoded_layers=False)
        pooled_output = self.dropout(pooled_output)
        #logits = self.classifier(pooled_output)
        return pooled_output



In [46]:
bert = BertForQuoraClassification.from_pretrained('bert-base-uncased')

In [47]:
class MyEnsemble(nn.Module):
    def __init__(self, bert):
        super(MyEnsemble, self).__init__()
        self.bert = bert
        self.classifier = nn.Linear(768*2, 512)
        self.fc1 = nn.Linear(512,256)
        self.fc2 = nn.Linear(256,1)
        
    def forward(self, x1, x2):
        x1 = self.bert(x1)
        x2 = self.bert(x2)
        x = torch.cat((x1, x2), dim=1)
        x = self.classifier(x)
        x = F.relu(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return F.sigmoid(x)


final = MyEnsemble(bert)

In [48]:
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(final,device_ids=(0,1,2))

model.to(device)

Let's use 3 GPUs!


DataParallel(
  (module): MyEnsemble(
    (bert): BertForQuoraClassification(
      (bert): BertModel(
        (embeddings): BertEmbeddings(
          (word_embeddings): Embedding(30522, 768)
          (position_embeddings): Embedding(512, 768)
          (token_type_embeddings): Embedding(2, 768)
          (LayerNorm): BertLayerNorm()
          (dropout): Dropout(p=0.1)
        )
        (encoder): BertEncoder(
          (layer): ModuleList(
            (0): BertLayer(
              (attention): BertAttention(
                (self): BertSelfAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                  (value): Linear(in_features=768, out_features=768, bias=True)
                  (dropout): Dropout(p=0.1)
                )
                (output): BertSelfOutput(
                  (dense): Linear(in_features=768, out_features=768, bias=True)
                  (

In [49]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = BertAdam(optimizer_grouped_parameters, lr=3e-5)

In [50]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    #pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    #acc = (np.abs(preds - labels_flat))/len(labels_flat)
    super_threshold_indices = preds > 0.5
    preds[super_threshold_indices] = 1
    super_threshold_indices = preds < 0.5
    preds[super_threshold_indices] = 0
    return np.sum(preds == labels_flat) / len(labels_flat)

In [51]:
!nvidia-smi

Wed Feb  6 00:11:01 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 387.26                 Driver Version: 387.26                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:06:00.0 Off |                  N/A |
| 35%   60C    P5    17W / 250W |   6923MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:09:00.0 Off |                  N/A |
| 31%   49C    P8    11W / 250W |   4869MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [35]:
max_grad_norm = 1.0
epoch = 5
for i in trange(epoch,desc='Epoch'):  
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for batch_idx, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids1,b_input_ids2, b_labels = batch
        optimizer.zero_grad()
        output = model(b_input_ids1,b_input_ids2)
        #loss_fct = CrossEntropyLoss()
        #loss = loss_fct(output.view(-1, 1), b_labels.view(-1))
        loss = F.binary_cross_entropy(output.view(-1), b_labels.float().view(-1))
        loss.backward()
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids1.size(0)
        nb_tr_steps += 1
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
        
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids1,b_input_ids2, val_labels = batch
        
        with torch.no_grad():
            logits = model(b_input_ids1,b_input_ids2)
            tmp_eval_loss = loss = F.binary_cross_entropy(logits.view(-1), val_labels.float().view(-1))
        logits = logits.detach().cpu().numpy()
        label_ids = val_labels.to('cpu').numpy()
        #predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        #predictions.append(logits)
        true_labels.append(label_ids)
        #preds = np.array(logits)
        #super_threshold_indices = preds > 0.5
        #preds[super_threshold_indices] = 1
        #super_threshold_indices = preds <= 0.5
        #preds[super_threshold_indices] = 0
            
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids1.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    #pred_tags = [for p in predictions for p_i in p]
    #valid_tags = [l in true_labels for l_i in l for l_ii in l_i]
    #print("F1-Score: {}".format(f1_score(preds, label_ids)))    

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]/users/aditya.sharma/super-resolution-master_new/SR_new/lib/python3.5/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train loss: 0.588601137342957


Epoch:  20%|██        | 1/5 [2:21:04<9:24:18, 8464.73s/it]

Validation loss: 0.5824119295641969
Validation Accuracy: 18.469872048442063
Train loss: 0.6460132952122667


Epoch:  40%|████      | 2/5 [4:41:35<7:02:43, 8454.41s/it]

Validation loss: 0.6592472347466252
Validation Accuracy: 20.17009493670886
Train loss: 0.6596534408949346


Epoch:  60%|██████    | 3/5 [7:02:28<4:41:48, 8454.03s/it]

Validation loss: 0.659038046162717
Validation Accuracy: 20.17009493670886


KeyboardInterrupt: 